# Clinical Feature Engineering
## Vital Signs Data
### Load and Filter Relevant Vitals from CHARTEVENTS
In this chapter, we first load CHARTEVENTS.csv, selecting only the columns necessary for analyzing vital signs. We filter for ITEMIDs corresponding to core physiological variables (heart rate, blood pressure, temperature, etc.), and assign a descriptive label to each one. This preprocessing step reduces the file's dimensionality and focuses the analysis on the most informative clinical measurements within the early hours of ICU stay.# Feature Engineering

In [46]:
import pandas as pd
import numpy as np
import os

RAW_PATH = "../data/raw/"
EXPORT_PATH = "../data/processed/"

# === Load ICU stay cohort with admission times ===
df_final = pd.read_csv(os.path.join(EXPORT_PATH, "df_final_static.csv"), parse_dates=["INTIME"])

In [47]:
chartevents = pd.read_csv(os.path.join(RAW_PATH, "CHARTEVENTS.csv"), usecols=["SUBJECT_ID", "ICUSTAY_ID", "ITEMID", "CHARTTIME", "VALUENUM"])
chartevents = chartevents[chartevents["VALUENUM"].notnull()]

vital_signs = {
    'HeartRate': [211, 220045],
    'SysBP': [51, 455],
    'DiasBP': [8368, 8441],
    'MeanBP': [52, 456],
    'RespRate': [618, 220210],
    'Temperature': [223761, 678],
    'SpO2': [646, 220277],
    'Glucose': [807, 220621]
}

itemid_to_vital = {itemid: name for name, ids in vital_signs.items() for itemid in ids}
chartevents = chartevents[chartevents["ITEMID"].isin(itemid_to_vital)]
chartevents["VITAL_TYPE"] = chartevents["ITEMID"].map(itemid_to_vital)

### Restrict to First 24 Hours of ICU Stay
At this stage, we restrict CHARTEVENTS observations to the first 24 hours after ICU admission. We compute the time difference in hours between CHARTTIME and INTIME, keeping only those records that fall within this range. This temporal alignment is essential to ensure that the clinical variables capture the patient's initial condition, which is key for accurate LOS prediction.

In [48]:
# Load ICU stay times to align chart events temporally
icu_times = df_final[["ICUSTAY_ID", "INTIME"]].drop_duplicates()
icu_times["INTIME"] = pd.to_datetime(icu_times["INTIME"])

# Convert chart times and merge INTIME
chartevents["CHARTTIME"] = pd.to_datetime(chartevents["CHARTTIME"])
chartevents = chartevents.merge(icu_times, on="ICUSTAY_ID", how="inner")

# Compute hours from ICU admission and filter
chartevents["HOURS_FROM_INTIME"] = (chartevents["CHARTTIME"] - chartevents["INTIME"]).dt.total_seconds() / 3600
chartevents = chartevents[(chartevents["HOURS_FROM_INTIME"] >= 0) & (chartevents["HOURS_FROM_INTIME"] <= 24)]

### Aggregate Vital Sign Statistics per ICU Stay
Here, we compute summary statistics for each vital sign within the first 24 hours of ICU stay. For each ICUSTAY_ID, we calculate the mean, standard deviation, minimum, and maximum values of every variable. These aggregations offer a compact yet expressive representation of initial vital sign dynamics, facilitating effective modeling of the target LOS.

In [49]:
# Aggregate: mean, std, min, max for each vital per ICU stay
agg_funcs = ["mean", "std", "min", "max"]
vitals_agg = chartevents.groupby(["ICUSTAY_ID", "VITAL_TYPE"])["VALUENUM"].agg(agg_funcs).unstack("VITAL_TYPE")

# Flatten column names
vitals_agg.columns = [f"{vital}_{stat}" for vital, stat in vitals_agg.columns]
vitals_agg = vitals_agg.reset_index()

# Merge with df_final
df_final = df_final.merge(vitals_agg, on="ICUSTAY_ID", how="left")

## Lab Tests Data
### Load and Filter Relevant Lab Tests
In this step, we load laboratory data (LABEVENTS) and select a subset of clinically relevant tests (e.g., creatinine, lactate, sodium). We merge with D_LABITEMS.csv to associate readable labels to each ITEMID. By filtering for tests of interest, we reduce dimensionality and focus the model on high-impact predictive variables.

In [50]:
labevents = pd.read_csv(os.path.join(RAW_PATH, "LABEVENTS.csv"), usecols=["SUBJECT_ID", "HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"])
labitems = pd.read_csv(os.path.join(RAW_PATH, "D_LABITEMS.csv"), usecols=["ITEMID", "LABEL"])

# Keep only non-null numeric lab results
labevents = labevents[labevents["VALUENUM"].notnull()]

# Merge to get human-readable labels
labevents = labevents.merge(labitems, on="ITEMID", how="left")

# Select common lab tests of interest
labs_of_interest = [
    "Hemoglobin", "Creatinine", "Glucose", "Lactate", "Sodium",
    "Potassium", "WBC", "Platelet Count", "BUN", "INR", "PT", "PTT"
]

labevents = labevents[labevents["LABEL"].isin(labs_of_interest)]

### Restrict to First 24 Hours of ICU Stay
We restrict laboratory test results to the first 24 hours after ICU admission. The join with ICUSTAY_ID allows us to align each lab test chronologically within the ICU context. This selection ensures that the features reflect the patient’s initial physiological state, which is key to early LOS prediction

In [51]:
# Merge INTIME from ICU data
icu_times = df_final[["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "INTIME"]].drop_duplicates()
icu_times["INTIME"] = pd.to_datetime(icu_times["INTIME"])

# Convert CHARTTIME and align with ICU time
labevents["CHARTTIME"] = pd.to_datetime(labevents["CHARTTIME"])
labevents = labevents.merge(icu_times, on=["SUBJECT_ID", "HADM_ID"], how="inner")

# Keep lab values within first 24h from ICU admission
labevents["HOURS_FROM_INTIME"] = (labevents["CHARTTIME"] - labevents["INTIME"]).dt.total_seconds() / 3600
labevents = labevents[(labevents["HOURS_FROM_INTIME"] >= 0) & (labevents["HOURS_FROM_INTIME"] <= 24)]

#### Aggregate Lab Test Statistics per ICU Stay
In the end, we aggregate lab values by computing mean, standard deviation, minimum, and maximum for each test. These summary statistics capture the variability and magnitude of initial biochemical changes, enabling a compact yet informative clinical representation for each patient.

In [52]:
# Compute aggregation (mean, std, min, max) for each lab test
agg_funcs = ["mean", "std", "min", "max"]
labs_agg = labevents.groupby(["ICUSTAY_ID", "LABEL"])["VALUENUM"].agg(agg_funcs).unstack("LABEL")

# Flatten column names
labs_agg.columns = [f"{label.replace(' ', '_')}_{stat}" for label, stat in labs_agg.columns]
labs_agg = labs_agg.reset_index()

# Merge with main dataset
df_final = df_final.merge(labs_agg, on="ICUSTAY_ID", how="left")

## Intravenous Administration Data
### Load and Filter Input Events from INPUTEVENTS_MV
We load INPUTEVENTS_MV.csv containing intravenous administration data (medications, fluids, nutrition). We keep only records with non-null AMOUNT values and merge item descriptions from D_ITEMS.csv for readability. This step enables meaningful semantic categorization of initial therapeutic inputs.

In [53]:
# Load input events data (Metavision version)
input_mv = pd.read_csv(os.path.join(RAW_PATH, "INPUTEVENTS_MV.csv"), usecols=[
    "SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "STARTTIME", "ITEMID", "AMOUNT"
])
input_mv = input_mv[input_mv["AMOUNT"].notnull()]
input_mv["STARTTIME"] = pd.to_datetime(input_mv["STARTTIME"])

# Load item descriptions
d_items = pd.read_csv(os.path.join(RAW_PATH, "D_ITEMS.csv"), usecols=["ITEMID", "LABEL"])
input_mv = input_mv.merge(d_items, on="ITEMID", how="left")

### Categorize and Filter Input Types
Then we categorize treatments using keyword-based rules applied to the LABEL field. Clinically relevant groups (e.g., saline, nutrition, diuretics) are created to standardize therapeutic inputs. This step enables quantitative analysis of treatment intensity within interpretable categories.

In [54]:
# Categorize inputs based on keywords in the label
def categorize_input(label):
    if pd.isnull(label): return "Unknown"
    label = label.lower()
    if "sodium chloride" in label or "saline" in label or "ns" in label:
        return "Saline"
    elif "dextrose" in label:
        return "Dextrose"
    elif "nutrition" in label or "tpn" in label or "enteral" in label:
        return "Nutrition"
    elif "furosemide" in label or "lasix" in label:
        return "Diuretic"
    elif "blood" in label or "plasma" in label:
        return "Blood_Product"
    elif "heparin" in label:
        return "Heparin"
    else:
        return "Other"

input_mv["INPUT_CATEGORY"] = input_mv["LABEL"].apply(categorize_input)

### Restrict to First 24 Hours and Aggregate Input Volume
So, we filter administered inputs to the first 24 hours and compute total volume per input category. Aggregating by ICUSTAY_ID enables quantification of the patient’s initial therapeutic exposure. This clinical enrichment introduces features that reflect early treatment intensity.

In [55]:
# Merge INTIME to compute time from ICU admission
icu_times = df_final[["ICUSTAY_ID", "INTIME"]].drop_duplicates()
icu_times["INTIME"] = pd.to_datetime(icu_times["INTIME"])
input_mv = input_mv.merge(icu_times, on="ICUSTAY_ID", how="inner")

# Filter to first 24h
input_mv["HOURS_FROM_INTIME"] = (input_mv["STARTTIME"] - input_mv["INTIME"]).dt.total_seconds() / 3600
input_mv = input_mv[(input_mv["HOURS_FROM_INTIME"] >= 0) & (input_mv["HOURS_FROM_INTIME"] <= 24)]

# Aggregate volume by input category and ICU stay
input_agg = input_mv.groupby(["ICUSTAY_ID", "INPUT_CATEGORY"])["AMOUNT"].sum().unstack("INPUT_CATEGORY")
input_agg = input_agg.fillna(0)
input_agg.columns = [f"Input_{col}_24h" for col in input_agg.columns]
input_agg = input_agg.reset_index()

# Merge with df_final
df_final = df_final.merge(input_agg, on="ICUSTAY_ID", how="left")

## Output Events Data
### Load and Prepare Output Events
First, we load output events such as urine output and surgical drains from OUTPUTEVENTS.csv. Only valid entries are retained, and descriptions are merged from D_ITEMS.csv. These variables are critical indicators of hemodynamic status and renal function during ICU stay.

In [ ]:
# Load output events
outputs = pd.read_csv(os.path.join(RAW_PATH, "OUTPUTEVENTS.csv"), usecols=[
    "SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "CHARTTIME", "ITEMID", "VALUE"
])
outputs = outputs[outputs["VALUE"].notnull()]
outputs["CHARTTIME"] = pd.to_datetime(outputs["CHARTTIME"])

# Load item labels
d_items = pd.read_csv(os.path.join(RAW_PATH, "D_ITEMS.csv"), usecols=["ITEMID", "LABEL"])
outputs = outputs.merge(d_items, on="ITEMID", how="left")

### Categorize Output Types
Observations are categorized into clinically meaningful groups (e.g., Urine, Gastric, Drain) based on keywords in the LABEL field. This classification standardizes physiological output representation, enabling consistent quantitative analyses across patients.

In [ ]:
def categorize_output(label):
    if pd.isnull(label): return "Unknown"
    label = label.lower()
    if "urine" in label:
        return "Urine"
    elif "drain" in label or "chest tube" in label:
        return "Drain"
    elif "emesis" in label or "vomit" in label:
        return "Emesis"
    elif "gastric" in label or "stomach" in label:
        return "Gastric"
    elif "stool" in label or "feces" in label:
        return "Stool"
    else:
        return "Other"

outputs["OUTPUT_CATEGORY"] = outputs["LABEL"].apply(categorize_output)

### Filter by First 24 Hours and Aggregate Volumes
In this last step, we restrict physiological outputs to the first 24 hours and calculate total volumes by category. Aggregation by ICUSTAY_ID yields a concise yet informative measure of early physiological output, serving as a proxy for the patient’s acute clinical status.

In [ ]:
# Merge ICU times to compute hours from admission
icu_times = df_final[["ICUSTAY_ID", "INTIME"]].drop_duplicates()
icu_times["INTIME"] = pd.to_datetime(icu_times["INTIME"])
outputs = outputs.merge(icu_times, on="ICUSTAY_ID", how="inner")

# Restrict to first 24 hours
outputs["HOURS_FROM_INTIME"] = (outputs["CHARTTIME"] - outputs["INTIME"]).dt.total_seconds() / 3600
outputs = outputs[(outputs["HOURS_FROM_INTIME"] >= 0) & (outputs["HOURS_FROM_INTIME"] <= 24)]

# Aggregate total output per category
output_agg = outputs.groupby(["ICUSTAY_ID", "OUTPUT_CATEGORY"])["VALUE"].sum().unstack("OUTPUT_CATEGORY")
output_agg = output_agg.fillna(0)
output_agg.columns = [f"Output_{col}_24h" for col in output_agg.columns]
output_agg = output_agg.reset_index()

# Merge with main dataset
df_final = df_final.merge(output_agg, on="ICUSTAY_ID", how="left")
display(df_final.head())
print(df_final.info())
# Save the final processed DataFrame
df_final.to_csv(os.path.join(EXPORT_PATH, "df_model_ready.csv"), index=False)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,AGE,GENDER,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,FIRST_CAREUNIT,LOS,...,Input_Heparin_24h,Input_Nutrition_24h,Input_Other_24h,Input_Saline_24h,Output_Drain_24h,Output_Emesis_24h,Output_Gastric_24h,Output_Other_24h,Output_Stool_24h,Output_Urine_24h
0,269,106296,206613,40,M,EMERGENCY,EMERGENCY ROOM ADMIT,Medicaid,MICU,3.2788,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,3850.0
1,275,129886,219649,82,M,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,CCU,7.1314,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,200.0,0.0,1370.0
2,292,179726,222505,57,F,URGENT,TRANSFER FROM HOSP/EXTRAM,Private,MICU,0.8854,...,NaN,NaN,NaN,NaN,0.0,0.0,2230.0,3500.0,0.0,695.0
3,305,194340,217232,76,F,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Medicare,SICU,2.4370,...,5002.0,0.0,3933.0,24.0,0.0,0.0,0.0,1240.0,0.0,0.0
4,323,143334,264375,57,M,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,MICU,3.0252,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,700.0,0.0,855.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3685 entries, 0 to 3684
Data columns (total 89 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   SUBJECT_ID               3685 non-null   int64         
 1   HADM_ID                  3685 non-null   int64         
 2   ICUSTAY_ID               3685 non-null   int64         
 3   AGE                      3685 non-null   int64         
 4   GENDER                   3685 non-null   object        
 5   ADMISSION_TYPE           3685 non-null   object        
 6   ADMISSION_LOCATION       3685 non-null   object        
 7   INSURANCE                3685 non-null   object        
 8   FIRST_CAREUNIT           3685 non-null   object        
 9   LOS                      3685 non-null   float64       
 10  HOSPITAL_EXPIRE_FLAG     3685 non-null   int64         
 11  INTIME                   3685 non-null   datetime64[ns]
 12  mean_Glucose_x           2000 non-

### 4.5 Comorbidity
Comorbidity refers to the presence of one or more additional medical conditions coexisting with a patient’s primary disease. For example, in a patient diagnosed with sepsis, the concurrent presence of diabetes (ICD-9 250) or chronic kidney disease (ICD-9 585) constitutes relevant clinical comorbidities. These conditions can significantly influence the clinical trajectory and outcomes, such as by increasing the length of stay in the Intensive Care Unit (LOS), which is the primary outcome variable in our study.

Therefore, comorbidities carry both clinical and predictive relevance, as they may contribute to prolonged ICU stays and elevate the risk of in-hospital mortality.

In [ ]:
diagnoses = pd.read_csv(os.path.join(RAW_PATH, "DIAGNOSES_ICD.csv"), usecols=["SUBJECT_ID", "HADM_ID", "ICD9_CODE"])
comorb_counts = diagnoses[
    (diagnoses['HADM_ID'].isin(df_final['HADM_ID'])) &
    (~diagnoses['ICD9_CODE'].astype(str).str.startswith('038'))
].groupby('HADM_ID')['ICD9_CODE'].nunique().reset_index(name='NUM_COMORBIDITIES')


# Drop existing NUM_COMORBIDITIES column if it exists to avoid merge conflict
if 'NUM_COMORBIDITIES' in df_final.columns:
    df_final = df_final.drop(columns=['NUM_COMORBIDITIES'])

# Merge comorbidity counts into the final DataFrame
df_final = df_final.merge(comorb_counts, on='HADM_ID', how='left')
df_final['NUM_COMORBIDITIES'] = df_final['NUM_COMORBIDITIES'].fillna(0).astype(int)
# Save the final DataFrame with comorbidity counts
df_final.to_csv(os.path.join(EXPORT_PATH, "df_model_ready.csv"), index=False)
# Display final DataFrame info
print(df_final.info())
# Display the first few rows of the final DataFrame
print(df_final.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3685 entries, 0 to 3684
Data columns (total 92 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   SUBJECT_ID               3685 non-null   int64         
 1   HADM_ID                  3685 non-null   int64         
 2   ICUSTAY_ID               3685 non-null   int64         
 3   AGE                      3685 non-null   int64         
 4   GENDER                   3685 non-null   object        
 5   ADMISSION_TYPE           3685 non-null   object        
 6   ADMISSION_LOCATION       3685 non-null   object        
 7   INSURANCE                3685 non-null   object        
 8   FIRST_CAREUNIT           3685 non-null   object        
 9   LOS                      3685 non-null   float64       
 10  HOSPITAL_EXPIRE_FLAG     3685 non-null   int64         
 11  INTIME                   3685 non-null   datetime64[ns]
 12  mean_Glucose_x           2000 non-